# Scapping Wikipedia

In this Lab you will load the table of postal codes and to transform the data into a pandas dataframe using beautifulsoup4 and match it with ohther Geographic information 

<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#Scrapping">Scrapping</a></li>
        <li><a href="#Transforming">Transforming And adding Geographic Information</a></li>
        <li><a href="#Clustering">Clustering</a></li>
    </ol>
</div>
<br>
<hr>

Lets load required libraries

In [46]:
pip install lxml html5lib beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [62]:
pip install geocoder

     |████████████████████████████████| 102kB 5.7MB/s ta 0:00:011
     |████████████████████████████████| 92kB 6.0MB/s  eta 0:00:01
     |████████████████████████████████| 829kB 17.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [48]:
import pandas as pd
import requests
import lxml.html as lh

<div id="Scrapping">
    <h2>Scrapping Wekipedia</h2> 
</div>

Lets download the dataset. To download the data, we will use beautifulsoup4

In [50]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
print(len(dfs))

3


We got 3 tables => we will choose the first table that wee need

<div id="Transforming">
    <h2>Transforming And adding Latitude & Longitude from Geocoder API</h2> 
</div>

In [66]:
#Get the first table
df = dfs[0]
# Write to csv to keep it in storage
df.to_csv('table.csv')
df = pd.read_csv('table.csv')
#Drop the first column not needed: Unamed: 0
df.drop("Unnamed: 0",axis=1,inplace=True)
#Get the shape of our data: 180 rows and 3 Columns
print(df.shape)
#Get firt 5 rows
df.head()

(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [67]:
df['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

As recommended in Submission we should delete rows with a not assigned Borough

In [68]:
df=df[df['Borough']!='Not assigned']
#Show the Shape again after cleaning Borough Column => 180-77=103 Rows and 3 Columns, it's right!
df.shape

(103, 3)

In [69]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Get the Geographic coordinates

In [70]:
import geocoder # import geocoder
def get_geocoder(postal_code_from_df):
     # initialize your variable to None
     lat_lng_coords = None
     # loop until you get the coordinates
     while(lat_lng_coords is None):
       g = geocoder.google('{}, Toronto, Ontario'.format(postal_code_from_df))
       lat_lng_coords = g.latlng
     latitude = lat_lng_coords[0]
     longitude = lat_lng_coords[1]
     return latitude,longitude

In [72]:
df['Latitude'], df['Longitude'] = zip(*df['Postal Code'].apply(get_geocoder))

In [87]:
print(df.shape)
df.head()

(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<div id="Clustering">
    <h2>Clustering</h2> 
</div>

### Visualizing Neighbours in a Map

In [95]:
# create map of Toronto using latitude and longitude values (from Internet or by using geopy library)
"""
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
"""

# OR just search them in internet
#Toronto Lat/Long
toro_longitude=-79.3831843
toro_latitude=43.653226

In [198]:
# create map zoomin on toronto
map = folium.Map(location=[toro_latitude, toro_longitude], zoom_start=11)

In [105]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

### To cluster these Neighbours we shoudd use Foursquare Database to get neighbours of each element in our dataset and then we will Cluster these Neighbours based on he Category

#### Define Foursquare Credentials and Version

In [106]:
CLIENT_ID = 'S1BCO3PQ40KKAMECPDFNEXKSTKZFAXLNRGA3L3UJ1NP0ENZV' # your Foursquare ID
CLIENT_SECRET = '4KYFOMWOWWLLKMLVDZPXZSOS3D1MDZTYKGC2BVOXJ5UKLEXA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S1BCO3PQ40KKAMECPDFNEXKSTKZFAXLNRGA3L3UJ1NP0ENZV
CLIENT_SECRET:4KYFOMWOWWLLKMLVDZPXZSOS3D1MDZTYKGC2BVOXJ5UKLEXA


In [111]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [112]:
#Getting Neighbours for each element in our dataset
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [115]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


### For Clustering K-Means Accept only numerical features so we should dummyfy Neighbourhoud VenueCategory 

In [119]:
#Let's chech how categories we have
print("Size of our dataset: ",toronto_venues.shape)
print('We have {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

Size of our dataset:  (1331, 7)
We have 238 uniques categories.


In [120]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']])

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Venue Category_Accessories Store,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,...,Venue Category_Train Station,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Video Store,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [121]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [123]:
toronto_grouped.head()

,Neighborhood,Venue Category_Accessories Store,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,...,Venue Category_Train Station,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Video Store,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0


In [125]:
X = toronto_grouped.drop('Neighborhood', 1).values
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [126]:
#Transforming our dataset
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

array([[-0.10314212, -0.14664712, -0.10314212, ..., -0.13617827,
        -0.1467456 , -0.33120955],
       [-0.10314212, -0.14664712, -0.10314212, ..., -0.13617827,
        -0.1467456 , -0.33120955],
       [-0.10314212, -0.14664712, -0.10314212, ..., -0.13617827,
        -0.1467456 , -0.33120955],
       [-0.10314212, -0.14664712, -0.10314212, ..., -0.13617827,
        -0.1467456 , -0.33120955],
       [-0.10314212, -0.14664712, -0.10314212, ..., -0.13617827,
         0.61713561, -0.33120955]])

Initialize KMeans with these parameters, where the output parameter is called k_means.

In [199]:
from sklearn.cluster import KMeans 
k_means = KMeans(init="k-means++", n_clusters=3, n_init=12)
k_means.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=12, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [200]:
k_means_labels = k_means.labels_
k_means_cluster_centers = k_means.cluster_centers_
k_means_labels

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [202]:
# add clustering labels to our fisrt dataset to have for each Neighbour the cluster predicted below
toronto_grouped.insert(0, 'Cluster Labels', k_means.labels_)

In [203]:
# merge toronto_grouped with our fisrt dataset df to add latitude/longitude for each neighborhood
df_merged=df
df_merged = df_merged.join(toronto_grouped.set_index('Neighborhood'), on='Neighborhood')
df_merged.head() # check Cluster Labels column!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Venue Category_Accessories Store,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,...,Venue Category_Train Station,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Video Store,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.033333
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,0.076923,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.000000
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.033333,0.0,0.033333


In [204]:
#Function to affect color to each cluster: used 5 colors to try different iterations 3,4,5 Clusters 
def color_producer(k_means_labels):
    if k_means_labels == 0:
        return 'green'
    elif k_means_labels == 1:
        return 'black'
    elif k_means_labels == 2:
        return 'yellow'
    elif k_means_labels == 3:
        return 'blue'
    else:
        return 'red'

In [205]:
# create map
map_clusters = folium.Map(location=[toro_latitude, toro_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill_color=color_producer(cluster),
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

## Thank you for reviewing my Humble work!